<a href="https://colab.research.google.com/github/Insights-Labs-Consultant-Agency/yelp-google-maps-reviews-and-recommendations/blob/data-pipeline/notebooks/4.5-fp-etl-glue-job-yelp-tip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción, transformación y carga (ETL)

En este notebook, nuestro objetivo es realizar el proceso de extracción, transformación y carga (ETL) de los datos de Yelp y Google Maps utilizando la librería AWS Glue 3.0 que servirá como base para los scripts de los diferentes ETL Glue Jobs que se usaran en AWS Glue Workflow en el proceso de carga al DW. En esta etapa, se realizará un proceso de limpieza previa y posterior normalización para construir un DER.

## 0 Configuraciones Globales e Importaciones

En esta sección,instalamos e importamos todas las librerías y/o módulos necesarios para nuestro proceso ETL y establecemos configuraciones globales de ser requerido.

### Instalación de librerías y/o Dependencias

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://aws-glue-etl-artifacts.s3.amazonaws.com/glue-common/apache-maven-3.6.0-bin.tar.gz
!tar xvf apache-maven-3.6.0-bin.tar.gz -C /bin/ > /dev/null
!wget -q https://aws-glue-etl-artifacts.s3.amazonaws.com/glue-3.0/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3.tgz
!tar xvf spark-3.1.1-amzn-0-bin-3.2.1-amzn-3.tgz -C /bin/ > /dev/null
!pip install -q findspark

### Exportación de Variables Entorno

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] += ":/bin/apache-maven-3.6.0/bin"
os.environ["SPARK_HOME"] = "/bin/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3/"
os.environ["SPARK_CONF_DIR"] = "/bin/aws-glue-libs/conf"

### Instalación de AWS GLue 3.0 Libs

In [ ]:
!git clone -b glue-3.0 https://github.com/awslabs/aws-glue-libs.git /bin/aws-glue-libs
!chmod +x /bin/aws-glue-libs/bin/glue-setup.sh
!bash /bin/aws-glue-libs/bin/glue-setup.sh > /dev/null
!cp -r /bin/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3/jars/netty-all-4.1.51.Final.jar /bin/aws-glue-libs/jarsv1/

Cloning into '/bin/aws-glue-libs'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 321 (delta 67), reused 67 (delta 50), pack-reused 217
Receiving objects: 100% (321/321), 160.42 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (205/205), done.
rm: cannot remove 'PyGlue.zip': No such file or directory
rm: cannot remove '/bin/aws-glue-libs/conf/spark-defaults.conf': No such file or directory


### Importación de Librerías y/o Módulos

In [ ]:
import sys
sys.path.extend(["/bin/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3/python","/bin/spark-3.1.1-amzn-0-bin-3.2.1-amzn-3/python/lib/py4j-0.10.9-src.zip","/bin/aws-glue-libs/PyGlue.zip"])

import findspark
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.functions import split, explode, monotonically_increasing_id

findspark.init()
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
spark

## 1 Extracción

En esta sección, extraemos los datasets de la fuente y los leemos como un DataFrame de PySpark.

In [ ]:
# Ruta al archivo JSON checkin
file_path = '/content/drive/MyDrive/data/raw/yelp/tip.json'

In [ ]:
# Lee el archivo JSON con PySpark
df = spark.read.json(file_path)

## 2 Transformación

En esta sección, realizamos la limpieza inicial de los datos y las transformaciones necesarias. Esto puede incluir la creación de nuevas columnas  la eliminación de duplicados o columnas innecesarias, la gestión de valores nulos o la corrección de tipos de datos.

In [ ]:
# Elimina las filas duplicadas
df = df.dropDuplicates()

In [ ]:
# Convierte la columna 'date' al formato datetime
df = df.withColumn('date', df['date'].cast('timestamp'))

In [ ]:
# Ordena el DataFrame por y 'date'
df = df.orderBy('date')

In [ ]:
# Agrega una columna 'id' con un ID único para cada registro
df = df.withColumn('tip_id', monotonically_increasing_id())

In [ ]:
# Cambia el nombre de la columna 'text' por 'tip'
df = df.withColumnRenamed('text', 'tip')

In [ ]:
# Reordena las columnas para que 'id' sea la primera columna
df = df.select('tip_id', 'user_id', 'business_id', 'tip', 'date', 'compliment_count')

In [ ]:
df.show()

+------+--------------------+--------------------+--------------------+-------------------+----------------+
|tip_id|             user_id|         business_id|                 tip|               date|compliment_count|
+------+--------------------+--------------------+--------------------+-------------------+----------------+
|     0|rCumu_NyXfbyq16cP...|cXSyVvOr9YRN9diDk...|Simply the best b...|2009-04-16 13:11:49|               0|
|     1|LT_JU6bY75H918eKn...|kLYhipAEvdT1ORflu...|Crunchy French to...|2009-04-16 17:15:29|               0|
|     2|sxZX1armKzagQxDnb...|gTC8IQ_i8zXytWSly...|Lunch specials ar...|2009-04-16 19:59:35|               0|
|     3|KI4UutWtai0UKZ6ZK...|P3bw4h4kCaMaYqGGN...|Low selection, quiet|2009-04-16 23:45:46|               0|
|     4|C14KW1jjKM8QlYfMy...|aUjJ_x1KuvWmqIErh...|Pretty good pad t...|2009-04-17 00:34:55|               0|
|     5|8IcTnw2hmu5xjElM1...|DU4y4jJPE96-hBN_j...|Tries to cater fo...|2009-04-17 02:03:49|               0|
|     6|osLu8P3SkCJ

In [ ]:
df.printSchema()

root
 |-- tip_id: long (nullable = false)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- tip: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- compliment_count: long (nullable = true)



## 3. Carga

Finalmente, en esta sección cargamos nuestros datos transformados en formato parquet a su destino correspondiente.

### Google Drive

In [ ]:
# Ruta al archivo Parquet local
file_path = '/content/drive/MyDrive/data/cleaned/yelp/tip.parquet'

# Escribe el DataFrame a un archivo Parquet localmente
df.write.parquet(file_path)


### S3

In [ ]:
# Convierte el DataFrame de Spark a un DynamicFrame de Glue
dyf = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")

In [ ]:
# Escribe el DynamicFrame a S3 en formato Parquet
glueContext.write_dynamic_frame.from_options(
    frame = dyf,
    connection_type = "s3",
    connection_options = {"path": "s3://ruta/al/bucket"},
    format = "parquet"
)